<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
#загружаем необходимые библиотеки
import pandas as pd

from matplotlib.pyplot import plot as plt

from numpy.random import RandomState

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_0 = pd.read_csv('/Users/amirk/Downloads/geo_data_0.csv')
    data_1 = pd.read_csv('/Users/amirk/Downloads/geo_data_1.csv')
    data_2 = pd.read_csv('/Users/amirk/Downloads/geo_data_2.csv')

In [ ]:
#напишем функцию для знакомства с датасетами:
def explore_data(data):
    display(data.head())
    print()
    print(data.info())
    print()
    print(data.describe())
    print()
    print('Количество полных дубликатов в данных равно ', data.duplicated().sum())
    print('Количество дубликатов в столбце "id" равно ', data['id'].duplicated().sum())
    print(data[data['id'].duplicated(keep=False)].sort_values(by='id'))
    data.hist(grid=True, figsize=(10,7))

In [ ]:
explore_data(data_0)

В датасете `data_0` пропусков и полных дубликатов нет, выбросов в данных нет. Выявлено 10 дубликатов в столбце **id**, данные в них различаются, вероятно ошибка при сборе данных. 

In [ ]:
explore_data(data_1)

В датасете `data_1` также нет пропусков и дубликатов, выбросов в данных нет.

In [ ]:
explore_data(data_2)

В датасете `data_2` также нет пропусков и дубликатов, выбросов в данных нет.

Предобработка данных
- в каждом датасете выявлены единичные дубликаты в столбце `id`. Удалим данные строки, т.к. они критически не повлияют на выборку в целом.
- в данных присутсвует столбец `id`, который не несет в себе информации необходимой для построения моделей. Удалим его.   
- данные количественные - семплирование проводить не нужно
- для загрузки данных в модель линейной регрессии и получения коэффициентов (возможно понадобятся в будущем), характеризующих значимость признаков, необходимо выполнить масштабирование признаков.

In [ ]:
#создадим функцию для предобработки данных
def prep_data(data):
    data_temp = data.drop_duplicates(subset='id').drop('id', axis=1)
    X = data_temp.drop('product', axis=1)
    y = data_temp['product']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test 


In [ ]:
data_0_samp = prep_data(data_0)
data_1_samp = prep_data(data_1)
data_2_samp = prep_data(data_2)

## Обучение и проверка модели

In [ ]:
def best_model(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    params = {}
    grid_model = GridSearchCV(model, 
                              params, 
                              cv=5, 
                              scoring='neg_root_mean_squared_error', 
                              error_score='raise', 
                              n_jobs=-1, 
                              verbose=1)
    grid_model.fit(X_train, y_train)
    y_predict = grid_model.predict(X_test)
    print(f'Средний запас сырья на месторождении в данном регионе {y_predict.mean():.3f} тыс. баррелей')
    print('Значение RMSE = ', mean_squared_error(y_test, y_predict, squared=False))
    return y_predict 

In [ ]:
y_predict0 = best_model(data_0_samp[0], data_0_samp[1], data_0_samp[2], data_0_samp[3])

In [ ]:
y_predict1 = best_model(data_1_samp[0], data_1_samp[1], data_1_samp[2], data_1_samp[3])

In [ ]:
y_predict2 = best_model(data_2_samp[0], data_2_samp[1], data_2_samp[2], data_2_samp[3])

**Промежуточные выводы**
- в регионе 1 и 3 предполагаемые запасы сырья на одной скважине схожи, 92.681 и 94.938 тыс. баррелей соответсвенно. При этом значения RMSE очень высокие 37.596 и 40.049 тыс. баррелей соответсвенно. Модель плохо предсказывает запас сырья.
- в регионе 2 предполагаемые запасы сырья на одной скважине 69.314 тыс. баррелей, что меньше, чем 1 и 3 регионе, однако RMSE равно 0.894 тыс. баррелей. Модель очень хорошо предсказывает данные.
- данные различия могут повлиять на потенциальную прибыль. Проверим какая тактика будет более выигрышной, разработка скажин с меньшим количеством запасов сырья, но зато стабильно одинаково, либо регионы с большим разбросом запасов на скважине, но с большим средниим запасом.

## Подготовка к расчёту прибыли

In [ ]:
# пропишем все необходимые переменные
EXPENSE = 10_000_000_000
EXPLORE_WELL = 500
BEST_WELL = 200
PROFIT = 450000 
min_barrel_per_well = EXPENSE / BEST_WELL / PROFIT

In [ ]:
print(f'Минимально количество сырья на скважине, необходимое для безубыточной добычи равно не \
        менее {min_barrel_per_well:.3f} тыс. баррелей.')

Минимальное количество сырья на скважине необходимое для безубыточной добычи гораздо выше, чем среднее количество предполагаемых запасов сырья: **111.111 тыс. баррелей** против **92.634, 69.364 и 95.12 тыс. баррелей** в 1, 2 и 3 регионеах соответсвенно. При этом, учитывая высокие значения RMSE в 1 и 3 регионах, лучшие скважины могут иметь необходимый минимальный объем сырья для безубыточной добычи.

## Расчёт прибыли и рисков 

In [ ]:
#калькулятор для расчета прибыли
def calc_profit(y_predict, y_test, a):
    if a == 1:
        y_predict = pd.Series(y_predict, index=y_test.index)
        y_predict_best200 = y_predict.sort_values(ascending=False)[:200]
        y_test_best200 = y_test.loc[y_predict_best200.index]
        profit_pred = y_predict_best200.sum() * PROFIT - EXPENSE
        profit_test =  y_test_best200.sum() * PROFIT - EXPENSE
        print(f'Предполагаемая прибыль от 200 скважин с максимальным количеством сырья равна {profit_pred:.3f}')
        print(f'Реальная прибыль от спрогнозированных 200 скважин с максимальным количеством сырья равна {profit_test:.3f} (расчет на тестовых данных)')
    else:
        y_predict_best200 = y_predict.sort_values(ascending=False)[:200]
        y_test_best200 = y_test.loc[y_predict_best200.index]
        profit_pred = y_test_best200.sum() * PROFIT - EXPENSE
        return profit_pred

In [ ]:
calc_profit(y_predict0, data_0_samp[3], 1)

In [ ]:
calc_profit(y_predict1, data_1_samp[3], 1)

In [ ]:
calc_profit(y_predict2, data_2_samp[3], 1)

In [ ]:
#калькулятор прибыли  после применения бутстрепа
def bootstrap_calc(y_predict, y_test):
    state = RandomState(123)
    values = []
    count = 0
    bootstrap_samples = 1000
    y_predict = pd.Series(y_predict, index=y_test.index)
    for i in range(bootstrap_samples):
        subsamples = y_predict.sample(EXPLORE_WELL, replace=True, random_state=state)
        values.append(calc_profit(subsamples, y_test, 0))
        if calc_profit(subsamples, y_test, 0) < 0:
            count += 1
    values = pd.Series(values)
    print(f'Средняя прогнозируемая прибыль на 200 скважинах с максимальным количеством сырья равна {int(values.mean())}, 95% доверительный интервал {int(values.quantile(q=0.025))} - {int(values.quantile(q=0.975))}. Риск убытков равен {count / bootstrap_samples:.2%}')
    return [int(values.mean()), int(values.quantile(q=0.025)), int(values.quantile(q=0.975)), format((count / bootstrap_samples), '.2%')]

In [ ]:
geo1 = bootstrap_calc(y_predict0, data_0_samp[3])

In [ ]:
geo2 = bootstrap_calc(y_predict1, data_1_samp[3])

In [ ]:
geo3 = bootstrap_calc(y_predict2, data_2_samp[3])

In [ ]:
#Соберем все в одну табличку 
final_table = pd.DataFrame(columns=['Средняя прибыль (руб.)', 'Нижняя граница 95% доверительного интервала', 'Верхняя граница 95% доверительного интервала', 'Риск убытков'])
for i in [geo1, geo2, geo3]:
    i = pd.DataFrame([i], columns=['Средняя прибыль (руб.)', 'Нижняя граница 95% доверительного интервала', 'Верхняя граница 95% доверительного интервала', 'Риск убытков'])
    final_table = pd.concat([final_table, i]) 
final_table.reset_index(inplace=True, drop=True)
final_table.rename(index={0:'geo1', 1:'geo2', 2:'geo3'}, inplace=True)
final_table.sort_values(by='Средняя прибыль (руб.)', ascending=False)

- в регионах 1 и 3 риск убытков составил более 2.5%, поэтому для разработки они не рассматриваются.     
    
- наибольшая средняя прибыль выявлена в регионе 2 -  488403884	руб.   

**Таким образом, лучшим регионом для разработки является регион 2.**

## Выводы

- в ходе исследования мы ознакомились с данными, выполнили подготовку данных для их загрузки в модели
- построили модель прогнозирования запаса сырья на отдельной скажине, посчитали прогнозируемый средний объем запаса сырья на 1 скважине в регионе.
   - в регионе 1 и 3 предполагаемые запасы сырья на одной скважине схожи, 92.681 и 94.938 тыс. баррелей соответсвенно. При этом значения RMSE очень высокие 37.596 и 40.049 тыс. баррелей соответсвенно.
   - в регионе 2 предполагаемые запасы сырья на одной скважине 69.314 тыс. баррелей, что меньше, чем 1 и 3 регионе, однако RMSE равно 0.894 тыс. баррелей.  
- рассчитали минимальный объем запаса сырья на скважине, требуемый для безубыточной добычи (111.111 тыс. баррелей)
- написали функцию для расчета предполагаемой прибыли от 200 скважин с максимальным количеством сырья.
- методикой бутстрепа рассчитали распределение прибыли, оценили среднюю прибыль, 95% доверительный интервал для прибыли, а также рсассчитали риск убытков. 

| Регион | Средняя прибыль (руб.) | Нижняя граница 95% доверительного интервала | Верхняя граница 95% доверительного интервала | Риск убытков |
|----|----|----|----|----|
| geo2 | 488403884 | 79510951 | 883658384 |	0.80% |
|geo1|	394841382|	-131716443|	900093498|	6.70%|
|geo3|	344163677|	-209593341|	840262477|	10.70%|
	   
      - в регионах 1 и 3 риск убытков составил более 2.5%, поэтому для разработки они не рассматриваются.     
      - наибольшая средняя прибыль выявлена в регионе 2 -  488403884	руб.
**Таким образом, лучшим регионом для разработки является регион 2.**